In [1]:
import ee
import pandas as pd
import folium
import matplotlib.pyplot as plt

ee.Initialize()



In [2]:
states = ee.FeatureCollection("users/mdm/india_soiStates")
punjab_polygon = ee.Geometry.Polygon([[[73.87088776, 29.54617119],
 [76.93139648, 29.54617119],
 [76.93139648, 32.57619095],
 [73.87088776, 32.57619095],
 [73.87088776, 29.54617119]]])

In [3]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)

def addYMD(image):
    date = ee.Number.parse(image.date().format('YYYYmmdd'))
    return image.addBands(ee.Image.constant(date).rename('date'))


In [5]:
folium.Map.add_ee_layer = add_ee_layer


In [6]:
firms = (ee.ImageCollection("FIRMS")
    .filterDate("2020-10-29", "2020-10-30")
    .select("T21", "confidence", "line_number")
    .map(addYMD)
    .mosaic())


In [7]:
projection = ee.ImageCollection("FIRMS").first().select("T21").projection()


In [8]:
table = firms.addBands(ee.Image.pixelLonLat()).sample(region= punjab_polygon, projection= projection)

In [9]:
print(table.getDownloadURL())

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/d260baaa1e12a642d1b25ec13a8c96d8-2d5f26b1b92b59816394f742ec561805:getFeatures


In [76]:

# Define a map centered on India
mapa = folium.Map(location=[22.369085, 79.893286], zoom_start=5)

fireVis = {
    "bands": ["T21"],
    "gamma": 0.95,
    "min": 300.0,
    "max": 509.29,
}
# Add the image layer to the map and display it.
#mapa.add_ee_layer(firms, fireVis, "FIRMS")

# Option 1

In [13]:
import os
print(os.getcwd())
india_shape = "../data/gadm/gadm36_IND_1.shp"
import geopandas as gpd
import matplotlib.pyplot as plt
import sys 
sys.path.insert(0, '../src/')
%matplotlib inline
india_gpd = gpd.read_file(india_shape)

# Extract a specific state of india by name
import extract_gadm_data as egd

punjab_gdf = egd.extract_state_dataframe(india_gpd, 'Punjab')
punjab_extents = egd.region_extents(punjab_gdf)

# Create bounding box for Punjab state and get the FIRMS collection for September 2021
punjab_bbox = egd.get_region_bounding_box(punjab_extents)



/Users/cmougan/Desktop/energy_harvest_trust/notebooks


In [12]:
[list(punjab_bbox[0]), list(punjab_bbox[1]), list(punjab_bbox[2]), list(punjab_bbox[3]), list(punjab_bbox[0])]

[[73.87088776, 29.54617119],
 [76.93139648, 29.54617119],
 [76.93139648, 32.57619095],
 [73.87088776, 32.57619095],
 [73.87088776, 29.54617119]]

In [78]:
firms = (ee.ImageCollection("FIRMS")
    .filterDate("2020-10-20", "2020-10-30")
    .select("T21", "confidence", "line_number")
    .map(addYMD)
    .mosaic())


In [79]:
projection = ee.ImageCollection("FIRMS").first().select("T21").projection()


In [80]:
table = firms.addBands(ee.Image.pixelLonLat()).sample(region= punjab_polygon, projection= projection)

In [81]:
print(table.getDownloadURL())

EEException: Expected a homogeneous image collection, but an image with an incompatible band was encountered. Mismatched type for band 'date':
Expected type: Integer<20200020, 20200020>.
  Actual type: Integer<20200021, 20200021>.
     Image ID: 2020295
This band might require an explicit cast.

In [51]:
firms.addBands(ee.Image.pixelLonLat()).sample(region= states, projection= projection).getDownloadURL()

EEException: Description length exceeds maximum.

In [28]:
table.getDownloadURL(f  ormat:'csv')

EEException: Expected a homogeneous image collection, but an image with an incompatible band was encountered. Mismatched type for band 'date':
Expected type: Integer<20210801, 20210801>.
  Actual type: Integer<20210802, 20210802>.
     Image ID: 2021214
This band might require an explicit cast.

In [27]:
print(table.getDownloadURL({
  format: "csv"
}))

AttributeError: 'dict' object has no attribute 'upper'

In [1]:
import pandas as pd

In [2]:
pwd

'/Users/cmougan/Desktop/energy_harvest_trust/notebooks'

In [4]:
df = pd.read_csv('../data/modis_2020_India.csv')

In [5]:
df

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,28.1282,96.9943,301.4,1.7,1.3,2020-01-01,357,Terra,MODIS,40,6.03,278.8,15.3,D,0
1,33.1827,74.0774,301.5,1.5,1.2,2020-01-01,534,Terra,MODIS,48,6.03,280.6,12.3,D,0
2,30.0068,80.6242,300.6,1.1,1.1,2020-01-01,535,Terra,MODIS,38,6.03,278.7,9.1,D,0
3,28.9858,79.6159,304.1,1.1,1.0,2020-01-01,535,Terra,MODIS,58,6.03,288.8,6.4,D,0
4,30.1529,80.2387,301.9,1.1,1.0,2020-01-01,535,Terra,MODIS,41,6.03,274.5,7.3,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76016,16.1899,75.4502,300.9,1.7,1.3,2020-12-31,1738,Terra,MODIS,35,6.03,289.5,9.0,N,0
76017,21.1035,72.6365,308.7,1.3,1.1,2020-12-31,1740,Terra,MODIS,75,6.03,291.7,13.7,N,2
76018,22.7952,86.2129,317.9,1.6,1.3,2020-12-31,2024,Aqua,MODIS,96,6.03,288.6,37.3,N,2
76019,20.9639,85.1806,306.5,1.5,1.2,2020-12-31,2025,Aqua,MODIS,68,6.03,289.9,16.4,N,2
